# Train and Test

In [1]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

# MLflow

In [2]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# Create a new MLflow Experiment - ambiente onde estarão registros de um dado projeto de ML
mlflow.set_experiment("primeiro_mlflow")

2024/03/03 16:12:32 INFO mlflow.tracking.fluent: Experiment with name 'primeiro_mlflow' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1709493152718, experiment_id='1', last_update_time=1709493152718, lifecycle_stage='active', name='primeiro_mlflow', tags={}>

In [3]:
# Start an MLflow run - um teste de modelo treinado
with mlflow.start_run(run_name='primeira'):
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature - registrar a entrada e a saída do modelo
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr, ### instancia do modelo treinado
        artifact_path="iris_model", ## pasta onde armazenara os aspectos desse modelo
        signature=signature, ### registros de signature
        input_example=X_train, ### exemplo de entrada 
        registered_model_name="tracking-quickstart", ### nome do modelo a ser registrado
    )

Successfully registered model 'tracking-quickstart'.
2024/03/03 16:26:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 1
Created version '1' of model 'tracking-quickstart'.


In [5]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

In [6]:
result

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1
4,6.8,2.8,4.8,1.4,1,1
5,5.4,3.4,1.5,0.4,0,0
6,5.6,2.9,3.6,1.3,1,1
7,6.9,3.1,5.1,2.3,2,2
8,6.2,2.2,4.5,1.5,1,1
9,5.8,2.7,3.9,1.2,1,1
